## Multivariate Time Series Analysis



In [1]:
from IPython.display import display, Markdown
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
plt.style.use('ggplot')

In [1]:
!pip install orbit-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 474 kB 5.6 MB/s 
     |████████████████████████████████| 722 kB 62.5 MB/s 
     |████████████████████████████████| 67.3 MB 115 kB/s 
     |████████████████████████████████| 11.2 MB 46.6 MB/s 
     |████████████████████████████████| 965 kB 64.2 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import orbit
from orbit.models import ETS, DLT
from orbit.diagnostics.plot import plot_predicted_data

In [3]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

### Import Data

In [4]:
dic_country = \
{'AUS': 'Australia',
 'BRA': 'Brazil',
 'CAN': 'India',
 'CHN': 'China',
 'GBR': 'United Kingdom',
 'IND': 'India',
 'JPN': 'Japan',
 'SGP': 'Singapore',
 'USA': 'United States'}
pd.DataFrame(dic_country,index=[0]).T.reset_index().rename(columns = {'index': 'country_code', 0: 'country_name'})

,country_code,country_name
0,AUS,Australia
1,BRA,Brazil
2,CAN,India
3,CHN,China
4,GBR,United Kingdom
5,IND,India
6,JPN,Japan
7,SGP,Singapore
8,USA,United States


In [ ]:
Data_dic_m = pd.ExcelFile('Manufacturing_filled.xlsx')
Data_dic_s = pd.ExcelFile('Service_filled.xlsx')

In [ ]:
def get_sheet_by_methods(data, method_num, verbose = 0):
  sheet_names = data.sheet_names
  sheet_imputation_map = pd.DataFrame([[s, s[:3].strip(),s[3:]] for s in sheet_names], columns = ['sheet_name', 'country_code', 'imputation method'])
  methods = sheet_imputation_map['imputation method'].unique()
  if verbose:
    print('methods tried:',methods, len(methods))
  return list(sheet_imputation_map.loc[sheet_imputation_map['imputation method'] == methods[method_num]]['sheet_name'].values)

In [ ]:
m_dfs = {}
for i in range(len(Data_dic_m.sheet_names) // 10):
  for sheet in get_sheet_by_methods(Data_dic_m, i):
    df = pd.read_excel(Data_dic_m, sheet_name=sheet)
    df['Year'] = [i.year for i in pd.to_datetime(df.Year, format='%Y')]
    # df = df.set_index('Year')
    m_dfs[(sheet[:3], sheet[3:].strip())] = df

In [ ]:
s_dfs = {}
for i in range(len(Data_dic_s.sheet_names) // 10):
  for sheet in get_sheet_by_methods(Data_dic_s, i):
    df = pd.read_excel(Data_dic_s, sheet_name=sheet)
    df['Year'] = [i.year for i in pd.to_datetime(df.Year, format='%Y')]
    # df = df.set_index('Year')
    s_dfs[(sheet[:3], sheet[3:].strip())] = df

In [ ]:
# way to retrieve method and country
method = ['Mean','Median','LOCF','NOCB','Rolling statistics']
country = list(dic_country.keys())

m_dfs[('CHN','Rolling statistics')] # stands for AUS mean data

,Year,Employment_in_industry_male,Research_development_expenditure,Researchers in R&D,Population_labor_rate,Population_density,Foreign_direct_investment,Air_freight_million_ton_km,Container_port_traffic_TEU,Industry_value_added_current_USD,GDP_per_capita_current_USD,Final_consumption,Employment_in_industry_%_of_total_employment,Manufacturing_value added_%_of_GDP
0,2004,23.020000,1.214980,700.070129,71.508852,137.518959,3.483641,7024.250000,7.472544e+07,8.975084e+11,1508.668098,1.080060e+12,22.500000,31.975070
1,2005,23.818524,1.265466,776.427218,71.864781,137.959601,4.065213,7325.815583,7.066211e+07,9.938809e+11,1641.619485,1.168687e+12,23.206177,32.039633
2,2006,24.698223,1.305919,830.551835,72.182206,138.409009,4.239218,7469.610307,7.621485e+07,1.117492e+12,1821.215001,1.277861e+12,23.988682,32.201601
3,2007,25.685230,1.327484,902.766581,72.437777,138.862164,4.290687,8653.317969,8.499997e+07,1.291569e+12,2098.931919,1.447364e+12,24.883263,32.259436
4,2008,26.430512,1.359998,977.903195,72.627705,139.325914,4.137760,9403.532641,9.349443e+07,1.529429e+12,2474.863985,1.677799e+12,25.519274,32.220995
5,2009,27.124445,1.435016,945.576872,72.769968,139.800020,3.751629,10036.776686,9.726143e+07,1.730059e+12,2808.940797,1.893683e+12,26.080607,32.067230
6,2010,27.857034,1.498120,931.769108,72.882186,140.283661,3.808671,11657.273715,1.051244e+08,1.979196e+12,3203.250677,2.139064e+12,26.673685,31.964344
7,2011,28.598025,1.557989,937.241400,72.956182,140.799189,3.787491,12740.789341,1.138893e+08,2.304714e+12,3714.737191,2.488820e+12,27.273254,31.985652
8,2012,29.347832,1.629334,952.767565,73.000673,141.380587,3.594014,13310.494187,1.230450e+08,2.621174e+12,4235.672876,2.866503e+12,27.883004,31.893573
9,2013,29.920328,1.700559,974.692446,73.016184,142.011581,3.486917,13840.677362,1.329691e+08,2.931702e+12,4773.862549,3.269090e+12,28.311481,31.657289


### Granger-Causality Test

In [ ]:
# TODO

### Train-test split

In [ ]:
def split_train_test(df, valid_num = 5):
    train = df[:-valid_num]
    valid = df[-valid_num:]
    return train, valid

train_manu = {}
valid_manu = {}
train_ser = {}
valid_ser = {}
for m in method:
  for con in country:
    train_manu[(con,m)], valid_manu[(con,m)]= split_train_test(m_dfs[(con,m)])
    train_ser[(con,m)], valid_ser[(con,m)]= split_train_test(s_dfs[(con,m)])

###  Exponential Smoothing (ETS)

#### Maximum a Posteriori (MAP)¶

In [ ]:
forecast_manu = {}
forecast_ser = {}
def fitting_ETS(m, con, response_col, date_col, result, train, test, seasonality = 5,estimator = 'stan-map'):
  ets = ETS(
    response_col=response_col,
    date_col=date_col,
    estimator= estimator,
    # seasonality=seasonality,
    seed=8888,
  )
  ets.fit(df=train[(con,m)])
  predicted_df = ets.predict(df=test[(con,m)])
  result[(con,m)] = predicted_df

In [ ]:
response_col = 'Manufacturing_value added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_ETS(m, con, response_col, date_col, forecast_manu, train_manu, valid_manu)

response_col = 'Services_value_added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_ETS(m, con, response_col, date_col, forecast_ser,train_ser, valid_ser)

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error

In [ ]:
m_metrics = {}
s_metrics = {}
for m in method:
  for con in country:
    if len(train_manu[(con,m)].columns) > 1 and 'Manufacturing_value added_%_of_GDP' in valid_manu[(con,m)].columns:
      y = valid_manu[(con,m)]['Manufacturing_value added_%_of_GDP']
      yhat = forecast_manu[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval =  [mae,rmse,mape]
    #print(mae, mse, rmse, con)
      m_metrics[(con,m)] = eval
    elif len(train_manu[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for manufaturce {con} {m} is non-stationary.')
      m_metrics[(con,m)] = [np.nan,np.nan,np.nan]

    if  len(train_ser[(con,m)].columns) > 1 and 'Services_value_added_%_of_GDP' in valid_ser[(con,m)].columns:
      y = valid_ser[(con,m)]['Services_value_added_%_of_GDP']
      yhat = forecast_ser[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval = [mae,rmse,mape]
      #print(mae, mse, rmse, con)
      s_metrics[(con,m)] = eval
    elif len(train_ser[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for service {con} {m} is non-stationary.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]

In [ ]:
m_eval = pd.DataFrame(m_metrics).transpose()
m_eval.columns = ['MAE', 'RMSE','MAPE']

m_eval

,,MAE,RMSE,MAPE
AUS,Mean,0.403926,0.414115,0.071370
BRA,Mean,0.608208,0.753682,0.061804
CAN,Mean,0.252489,0.274816,0.025653
CHN,Mean,0.795722,1.030792,0.029774
GBR,Mean,0.167354,0.188705,0.019057
IND,Mean,0.936837,1.125207,0.067893
JPN,Mean,0.267772,0.304826,0.013252
SGP,Mean,2.448069,2.606999,0.120876
USA,Mean,0.446287,0.454971,0.039855
AUS,Median,0.403926,0.414115,0.071370


In [ ]:
s_eval = pd.DataFrame(s_metrics).transpose()
s_eval.columns = ['MAE','RMSE','MAPE']
s_eval

,,MAE,RMSE,MAPE
AUS,Mean,1.828189,1.883519,0.027603
BRA,Mean,1.010412,1.738804,0.016789
CAN,Mean,1.716064,1.904473,0.025519
CHN,Mean,1.236163,1.404015,0.022913
GBR,Mean,0.565637,0.804392,0.007844
IND,Mean,0.773075,1.140966,0.015645
JPN,Mean,0.284643,0.303346,0.004097
SGP,Mean,0.764876,0.884622,0.010894
USA,Mean,0.968015,1.568313,0.012213
AUS,Median,1.828189,1.883519,0.027603


#### MCMC - Full Bayesian Sampling¶

In [ ]:
response_col = 'Manufacturing_value added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_ETS(m, con, response_col, date_col, forecast_manu, train_manu, valid_manu,5, 'stan-mcmc')

response_col = 'Services_value_added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_ETS(m, con, response_col, date_col, forecast_ser,train_ser, valid_ser,5 ,'stan-mcmc')

In [ ]:
m_metrics = {}
s_metrics = {}
for m in method:
  for con in country:
    if len(train_manu[(con,m)].columns) > 1 and 'Manufacturing_value added_%_of_GDP' in valid_manu[(con,m)].columns:
      y = valid_manu[(con,m)]['Manufacturing_value added_%_of_GDP']
      yhat = forecast_manu[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval =  [mae,rmse,mape]
    #print(mae, mse, rmse, con)
      m_metrics[(con,m)] = eval
    elif len(train_manu[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for manufaturce {con} {m} is non-stationary.')
      m_metrics[(con,m)] = [np.nan,np.nan,np.nan]

    if  len(train_ser[(con,m)].columns) > 1 and 'Services_value_added_%_of_GDP' in valid_ser[(con,m)].columns:
      y = valid_ser[(con,m)]['Services_value_added_%_of_GDP']
      yhat = forecast_ser[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval = [mae,rmse,mape]
      #print(mae, mse, rmse, con)
      s_metrics[(con,m)] = eval
    elif len(train_ser[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for service {con} {m} is non-stationary.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]

In [ ]:
m_eval = pd.DataFrame(m_metrics).transpose()
m_eval.columns = ['MAE', 'RMSE','MAPE']

m_eval

,,MAE,RMSE,MAPE
AUS,Mean,0.435871,0.455018,0.077077
BRA,Mean,0.799052,0.912611,0.080467
CAN,Mean,0.187334,0.235803,0.018992
CHN,Mean,1.027950,1.231088,0.038306
GBR,Mean,0.211469,0.226436,0.024021
IND,Mean,1.057400,1.225233,0.076371
JPN,Mean,0.303911,0.324366,0.014958
SGP,Mean,2.271355,2.431776,0.112069
USA,Mean,0.476224,0.496754,0.042553
AUS,Median,0.435871,0.455018,0.077077


In [ ]:
s_eval = pd.DataFrame(s_metrics).transpose()
s_eval.columns = ['MAE','RMSE','MAPE']
s_eval

,,MAE,RMSE,MAPE
AUS,Mean,1.684221,1.754009,0.025437
BRA,Mean,1.176198,1.868013,0.019467
CAN,Mean,1.748684,1.985079,0.025984
CHN,Mean,1.233062,1.441011,0.022837
GBR,Mean,0.601811,0.846980,0.008351
IND,Mean,0.798373,1.127706,0.016186
JPN,Mean,0.403019,0.442020,0.005801
SGP,Mean,0.763684,0.910472,0.010909
USA,Mean,1.006878,1.544071,0.012727
AUS,Median,1.684221,1.754009,0.025437


### Damped Local Trend (DLT)

#### Linear trends

In [ ]:

forecast_manu = {}
forecast_ser = {}
def fitting_DLT(m, con, response_col, date_col, result, train, test, seasonality = 5,estimator = 'stan-map',global_trend_option = 'linear'):
  dlt = DLT(
    response_col=response_col,
    date_col=date_col,
    estimator='stan-map',
    seed=8888,
    global_trend_option= global_trend_option,
    # for prediction uncertainty
    n_bootstrap_draws=1000,
)
  dlt.fit(df=train[(con,m)])
  predicted_df = dlt.predict(df=test[(con,m)])
  result[(con,m)] = predicted_df



In [ ]:
response_col = 'Manufacturing_value added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_DLT(m, con, response_col, date_col, forecast_manu, train_manu, valid_manu)

response_col = 'Services_value_added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_DLT(m, con, response_col, date_col, forecast_ser,train_ser, valid_ser)

In [ ]:
m_metrics = {}
s_metrics = {}
for m in method:
  for con in country:
    if len(train_manu[(con,m)].columns) > 1 and 'Manufacturing_value added_%_of_GDP' in valid_manu[(con,m)].columns:
      y = valid_manu[(con,m)]['Manufacturing_value added_%_of_GDP']
      yhat = forecast_manu[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval =  [mae,rmse,mape]
    #print(mae, mse, rmse, con)
      m_metrics[(con,m)] = eval
    elif len(train_manu[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for manufaturce {con} {m} is non-stationary.')
      m_metrics[(con,m)] = [np.nan,np.nan,np.nan]

    if  len(train_ser[(con,m)].columns) > 1 and 'Services_value_added_%_of_GDP' in valid_ser[(con,m)].columns:
      y = valid_ser[(con,m)]['Services_value_added_%_of_GDP']
      yhat = forecast_ser[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval = [mae,rmse,mape]
      #print(mae, mse, rmse, con)
      s_metrics[(con,m)] = eval
    elif len(train_ser[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for service {con} {m} is non-stationary.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]

In [ ]:
m_eval = pd.DataFrame(m_metrics).transpose()
m_eval.columns = ['MAE', 'RMSE','MAPE']

m_eval

,,MAE,RMSE,MAPE
AUS,Mean,0.484157,0.589798,0.086064
BRA,Mean,0.215617,0.237081,0.020968
CAN,Mean,0.860738,0.959935,0.087372
CHN,Mean,0.981321,1.142583,0.035782
GBR,Mean,0.560993,0.646505,0.063815
IND,Mean,0.765286,0.953864,0.055641
JPN,Mean,0.651426,0.688812,0.032074
SGP,Mean,3.271280,3.480848,0.161617
USA,Mean,0.166076,0.199986,0.014789
AUS,Median,0.484157,0.589798,0.086064


In [ ]:
s_eval = pd.DataFrame(s_metrics).transpose()
s_eval.columns = ['MAE','RMSE','MAPE']
s_eval

,,MAE,RMSE,MAPE
AUS,Mean,3.098628,3.290944,0.046817
BRA,Mean,3.629694,4.421576,0.059269
CAN,Mean,1.497154,1.651440,0.022274
CHN,Mean,2.201681,2.552600,0.041047
GBR,Mean,0.755836,0.794859,0.010605
IND,Mean,0.920976,1.128289,0.019009
JPN,Mean,0.813734,0.896699,0.011711
SGP,Mean,1.307784,1.586028,0.018747
USA,Mean,0.811014,1.085727,0.010303
AUS,Median,3.098628,3.290944,0.046817


### Local Global Trend (LGT)

In [ ]:
from orbit.models import LGT

In [ ]:

forecast_manu = {}
forecast_ser = {}
def fitting_LGT(m, con, response_col, date_col, result, train, test, seasonality = 5,estimator = 'stan-map'):
  lgt = LGT(
    response_col=response_col,
    date_col=date_col,
    estimator='stan-map',
    seed=8888,
)
  lgt.fit(df=train[(con,m)])
  predicted_df = lgt.predict(df=test[(con,m)])
  result[(con,m)] = predicted_df



In [ ]:
response_col = 'Manufacturing_value added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_DLT(m, con, response_col, date_col, forecast_manu, train_manu, valid_manu)

response_col = 'Services_value_added_%_of_GDP'
date_col = 'Year'
for con in country:
  for m in method:
    fitting_DLT(m, con, response_col, date_col, forecast_ser,train_ser, valid_ser)

In [ ]:
m_metrics = {}
s_metrics = {}
for m in method:
  for con in country:
    if len(train_manu[(con,m)].columns) > 1 and 'Manufacturing_value added_%_of_GDP' in valid_manu[(con,m)].columns:
      y = valid_manu[(con,m)]['Manufacturing_value added_%_of_GDP']
      yhat = forecast_manu[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval =  [mae,rmse,mape]
    #print(mae, mse, rmse, con)
      m_metrics[(con,m)] = eval
    elif len(train_manu[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for manufaturce {con} {m} is non-stationary.')
      m_metrics[(con,m)] = [np.nan,np.nan,np.nan]

    if  len(train_ser[(con,m)].columns) > 1 and 'Services_value_added_%_of_GDP' in valid_ser[(con,m)].columns:
      y = valid_ser[(con,m)]['Services_value_added_%_of_GDP']
      yhat = forecast_ser[(con,m)]['prediction']
      mae = mean_absolute_error(y, yhat)
      mape = mean_absolute_percentage_error(y, yhat)
      rmse = sqrt(mean_squared_error(y, yhat))
      eval = [mae,rmse,mape]
      #print(mae, mse, rmse, con)
      s_metrics[(con,m)] = eval
    elif len(train_ser[(con,m)].columns) == 1:
      print(f'Only contains target variable for service {con} {m}.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]
    else:
      print(f'Target variable for service {con} {m} is non-stationary.')
      s_metrics[(con,m)] = [np.nan,np.nan,np.nan]

In [ ]:
m_eval = pd.DataFrame(m_metrics).transpose()
m_eval.columns = ['MAE', 'RMSE','MAPE']

m_eval

,,MAE,RMSE,MAPE
AUS,Mean,0.484157,0.589798,0.086064
BRA,Mean,0.215617,0.237081,0.020968
CAN,Mean,0.860738,0.959935,0.087372
CHN,Mean,0.981321,1.142583,0.035782
GBR,Mean,0.560993,0.646505,0.063815
IND,Mean,0.765286,0.953864,0.055641
JPN,Mean,0.651426,0.688812,0.032074
SGP,Mean,3.271280,3.480848,0.161617
USA,Mean,0.166076,0.199986,0.014789
AUS,Median,0.484157,0.589798,0.086064


In [ ]:
s_eval = pd.DataFrame(s_metrics).transpose()
s_eval.columns = ['MAE','RMSE','MAPE']
s_eval

,,MAE,RMSE,MAPE
AUS,Mean,3.098628,3.290944,0.046817
BRA,Mean,3.629694,4.421576,0.059269
CAN,Mean,1.497154,1.651440,0.022274
CHN,Mean,2.201681,2.552600,0.041047
GBR,Mean,0.755836,0.794859,0.010605
IND,Mean,0.920976,1.128289,0.019009
JPN,Mean,0.813734,0.896699,0.011711
SGP,Mean,1.307784,1.586028,0.018747
USA,Mean,0.811014,1.085727,0.010303
AUS,Median,3.098628,3.290944,0.046817
